# correlated_words

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import TfidfVectorizer

from sites.bookcave import bookcave

In [ ]:
min_len, max_len = 250, 7500

In [ ]:
inputs, Y, categories, category_levels =\
    bookcave.get_data({'tokens'},
                      min_len=min_len,
                      max_len=max_len)
text_paragraph_tokens = [paragraph_tokens for paragraph_tokens, _ in inputs['tokens']]
len(text_paragraph_tokens)

In [ ]:
text_all_tokens = []
for paragraph_tokens in text_paragraph_tokens:
    all_tokens = []
    for tokens in paragraph_tokens:
        all_tokens.extend(tokens)
    text_all_tokens.append(all_tokens)
len(text_all_tokens)

Vectorize.

In [ ]:
min_gram = 1
max_gram = 1
max_features = 8192

In [ ]:
def identity(v):
    return v

In [ ]:
vectorizer = TfidfVectorizer(
    preprocessor=identity,
    tokenizer=identity,
    analyzer='word',
    token_pattern=None,
    ngram_range=(min_gram, max_gram),
    max_features=max_features,
    norm='l2',
    sublinear_tf=True)

In [ ]:
X = vectorizer.fit_transform(text_all_tokens)

In [ ]:
features = vectorizer.get_feature_names()

See [Multi Class Text Classification article](https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f).

In [ ]:
top_n = 256

In [ ]:
category_terms = []
for category_i, category in enumerate(categories):
    y = Y[category_i]
    scores, pvals = chi2(X, y)
    indices = np.argsort(scores)
    terms = [(features[indices[-1 - i]], scores[indices[-1 - i]]) for i in range(top_n)]
    category_terms.append(terms)

In [ ]:
for category_i, category in enumerate(categories):
    fname = 'words-{:d}_{}-{:d}-{:d}g{:d}-{:d}f-{:d}n.txt'.format(category_i,
                                                                  category,
                                                                  len(text_all_tokens),
                                                                  min_gram,
                                                                  max_gram,
                                                                  max_features,
                                                                  top_n)
    path = os.path.join('logs', 'correlated_words', fname)
    if not os.path.exists(path):
        terms = category_terms[category_i]
        with open(path, 'w', encoding='utf-8') as fd:
            fd.write('{:d}\n'.format(len(terms)))
            for term, score in terms:
                fd.write('{}\t{}\n'.format(term, score))

In [ ]:
category_terms[bookcave.CATEGORY_INDEX_PROFANITY]